<a href="https://colab.research.google.com/github/arjasc5231/cs470-team21/blob/main/preprocess_rationale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.8 MB/s 
     |████████████████████████████████| 7.6 MB 34.2 MB/s 
     |████████████████████████████████| 182 kB 34.1 MB/s 


In [7]:
import gzip
import json
import csv
import pandas as pd
import re
import torch
from transformers import BertTokenizer

# drive mount. colab에 내 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
sent = "Not as great as I remembered it.  It ₩ doesn't ₩ keep my 2 year old's attention. Maybe when he is older he will like it more,."
encoded_dict = tokenizer.encode_plus(
                          sent, 
                          add_special_tokens = True,
                          max_length = 512,
                          truncation=True,
                          padding = True,
                          return_attention_mask = True,
                          return_tensors = 'pt',
                    )
input_ids = encoded_dict['input_ids']
tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print(tokens)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

['[CLS]', 'not', 'as', 'great', 'as', 'i', 'remembered', 'it', '.', 'it', '₩', 'doesn', "'", 't', '₩', 'keep', 'my', '2', 'year', 'old', "'", 's', 'attention', '.', 'maybe', 'when', 'he', 'is', 'older', 'he', 'will', 'like', 'it', 'more', ',', '.', '[SEP]']


In [ ]:
# def combine_subtokens(tokens, relevance_score):
#     '''
#     combine subtokens to one word
#     '''

#     token = ['[CLS]', '[SEP]', '[MASK]', '[UNK]', '[PAD]']

#     for i in range(len(tokens)):
#         if tokens[i] in token: continue
        
#         num = 1
#         word = tokens[i]
#         score = relevance_score[i]
#         while(tokens[i+1][0] == '#'):
#             word += tokens[i+1][2:]
#             score += relevance_score[i+1]
#             num += 1
#             if tokens[i][0] != '#':
#                 tokens[i] = "##" + tokens[i]
#             i += 1
        
#         tokens.append(word)
#         # average the relevance scores of subtokens
#         relevance_score.append(score/num)

#     # delete subtokens
#     word, score = [], []
#     for i in range(len(tokens)):
#         if tokens[i][0] != '#':
#             word.append(tokens[i])
#             score.append(relevance_score[i])
    
#     return word, score


In [9]:
path = "/content/drive/MyDrive/CS470_team_2in1/dataset/Books_5.json.gz"

In [10]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield json.loads(l)

    
load_num = 0
load_total_num = 10000

parsed = []

for elt in parse(path):
    parsed.append(elt)
    load_num += 1
    if load_num == load_total_num: break

In [12]:
with gzip.open(path) as file:
    for i in range(3):
        print(file.readline())

b'{"overall": 5.0, "verified": false, "reviewTime": "03 30, 2005", "reviewerID": "A1REUF3A1YCPHM", "asin": "0001713353", "style": {"Format:": " Hardcover"}, "reviewerName": "TW Ervin II", "reviewText": "The King, the Mice and the Cheese by Nancy Gurney is an excellent children\'s book.  It is one that I well remember from my own childhood and purchased for my daughter who loves it.\\n\\nIt is about a king who has trouble with rude mice eating his cheese. He consults his wise men and they suggest cats to chase away the mice. The cats become a nuisance, so the wise men recommend the king bring in dogs to chase the cats away.  The cycle goes on until the mice are finally brought back to chase away the elephants, brought in to chase away the lions that\'d chased away the dogs.\\n\\nThe story ends in compromise and friendship between the mice and the king.  The story also teaches cause and effect relationships.\\n\\nThe pictures that accompany the story are humorous and memorable.  I was th

In [16]:
fields = ['sentence index', 'reviewText', 'overall', 'productID', 'positive rationale', 'negative rationale']

with open("/content/drive/MyDrive/CS470_team_2in1/dataset/preprocessed.csv", 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames= fields)

    writer.writeheader()
    for i in range(100):
        writer.writerow({'sentence index': i, 'reviewText': parsed[i]['reviewText'], 'overall': parsed[i]['overall'], 'productID': parsed[i]['asin'], 'positive rationale': parsed[i]['reviewText'], 'negative rationale': parsed[i]['reviewText']})


In [17]:
fields = ['index', 'sentence index', 'reviewText', 'overall', 'productID', 'positive rationale', 'negative rationale']

with open("/content/drive/MyDrive/CS470_team_2in1/dataset/preprocessed_balanced.csv", 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames= fields)

    count = [0, 0, 0, 0, 0, 0]
    num = 0
    writer.writeheader()
    for i in range(10000):
        if(i == 5 or i == 197): continue
        try:
            rating = int(parsed[i]['overall'])
            length = parsed[i]['reviewText'].count(" ") + 1
            # print(length)
        except:
            continue
        if(count[rating] >= 20): continue
        if(length >= 150): continue
        writer.writerow({'index': num, 'sentence index': i, 'reviewText': parsed[i]['reviewText'], 'overall': parsed[i]['overall'], 'productID': parsed[i]['asin'], 'positive rationale': parsed[i]['reviewText'], 'negative rationale': parsed[i]['reviewText']})
        num += 1
        count[rating] += 1

    print(count)

[0, 20, 20, 20, 20, 20]


In [19]:
# encoding
with open("/content/drive/MyDrive/CS470_team_2in1/dataset/preprocessed_balanced_working.csv", 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    cnt = 0
    pos_rat_en = []
    neg_rat_en = []
    for i, row in enumerate(csvreader):
        if(i==0): continue
        # if(i==3): break
        pos_temp = []
        flag = 0

        sent = row[5]
        encoded_dict = tokenizer.encode_plus(
                                sent, 
                                add_special_tokens = True,
                                max_length = 512,
                                truncation=True,
                                padding = True,
                                return_attention_mask = True,
                                return_tensors = 'pt',
                            )
        input_ids = encoded_dict['input_ids']
        tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())

        for tkn in tokens: # positive rationale
            if(tkn == '\\'): # start or end of rationale
                if(flag != 0): # start of rationale
                    flag = 0
                    continue
                elif(flag == 0): # end of rationale
                    flag = 1
                    continue
            elif(flag == 1): # middle of rationale
                pos_temp.append(1) 
            else: pos_temp.append(0) # not a rationale
        pos_rat_en.append(pos_temp)

        neg_temp = []
        flag = 0

        sent = row[6]
        encoded_dict = tokenizer.encode_plus(
                                sent, 
                                add_special_tokens = True,
                                max_length = 512,
                                truncation=True,
                                padding = True,
                                return_attention_mask = True,
                                return_tensors = 'pt',
                            )
        input_ids = encoded_dict['input_ids']
        tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())

        for tkn in tokens: # negative rationale
            if(tkn == '\\'): # start or end of rationale
                if(flag != 0): # start of rationale
                    flag = 0
                    continue
                elif(flag == 0): # end of rationale
                    flag = 1
                    continue
            elif(flag == 1): # middle of rationale
                neg_temp.append(1) 
            else: neg_temp.append(0) # not a rationale
        neg_rat_en.append(neg_temp)
    
    csvfile.seek(0)
    
    
    fields = ['index', 'sentence index', 'reviewText', 'overall', 'productID', 'positive rationale', 'negative rationale', 'positive rationale encoded', 'negative rationale encoded']

    with open("/content/drive/MyDrive/CS470_team_2in1/dataset/preprocessed_balanced_encoded.csv", 'w') as csvfile2:
        writer = csv.DictWriter(csvfile2, fieldnames= fields)

        writer.writeheader()
        for idx, row in enumerate(csvreader):
            if(idx == 0): continue
            writer.writerow({'index': row[0], 'sentence index': row[1], 'reviewText': row[2], 'overall': row[3], 'productID': row[4], 'positive rationale': row[5], 'negative rationale': row[6], 'positive rationale encoded': pos_rat_en[idx-1], 'negative rationale encoded': neg_rat_en[idx-1]})
            print(idx, row)
        
            

1 ['0', '1', 'The kids loved it!', '5', '1713353', 'The kids \\ loved \\ it!', 'The kids loved it!']
2 ['1', '2', 'My students (3 & 4 year olds) loved this book!  Definitely recommend it to other teachers.', '5', '1713353', 'My students (3 & 4 year olds) \\ loved \\ this book!  \\ Definitely \\ recommend it to other teachers.', 'My students (3 & 4 year olds) loved this book!  Definitely recommend it to other teachers.']
3 ['2', '3', 'LOVE IT', '5', '1713353', '\\ LOVE \\ IT', 'LOVE IT']
4 ['3', '4', 'Great!', '5', '1713353', '\\ Great \\ !', 'Great!']
5 ['4', '6', "I remember reading my parents' copy of this book until it fell apart. The story still makes me giggle. I just wanted to share it with my kids, who appreciated it even though most of them are teenagers now. The story is light-hearted and silly, but with a very powerful message about learning to live with the problems you have instead of trading them for larger problems.\n\nThe artwork is hysterical, too.", '5', '1713353', "I 

In [ ]:
# a = "The kids ((( loved ))) it!"
# b = []
# flag = 0
# print(a.split())
# for word in a.split():
#     if(word == "((("): 
#         b.append(1)
#         flag = 2
#     elif(flag == 2 or flag == 1):
#         flag -= 1
#         continue 
#     else: b.append(0)
# print(b)

['The', 'kids', '(((', 'loved', ')))', 'it!']
[0, 0, 1, 0]
